In [1]:
from pulp import *
import pandas as pd

## problem1

In [2]:
probA=LpProblem("shoes", LpMinimize)

In [3]:
#index
customer = ['STL1', 'STL2','STL3', 'STL4', 'KS1', 'KS2','COL','SPR']
plant = ['STL','KA','CO','SP','IN']

#parameters
d_shoes = [8000,6500,7500,0,9500,2300,7000,6000]
shoes_demand = dict(zip(customer,d_shoes))

d_boots = [2200,0,1200,900,4500,0,2000,3200]
boots_demand = dict(zip(customer,d_boots))

vc_shoes = [20,18,15,15,12]
shoes_vc = dict(zip(plant,vc_shoes))

vc_boots = [30,25,20,20,18]
boots_vc = dict(zip(plant,vc_boots))


trans_dis = [[10,10,10,10,250,250,120,215],\
             [250,250,250,250,10,10,130,165],\
             [120,120,120,120,130,130,10,170],\
             [215,215,215,215,165,165,170,10],\
             [240,240,240,240,15,15,125,170]]
df_distance = makeDict([plant,customer],trans_dis)

prod_line = [1,2]
fc = [[1000000,1700000],[1000000,1500000],[1000000,1500000],[900000,1300000]]
df_fc = makeDict([plant[1:],prod_line],fc)


#decision variables
y_vars = LpVariable.dicts("y",(plant[1:],prod_line),lowBound=0,cat='Binary')
shoes_vars = LpVariable.dicts("shoes",(plant,customer),lowBound=0,cat='Continuous')
boots_vars = LpVariable.dicts("boots",(plant,customer),lowBound=0,cat='Continuous')

shoes_produced = {}
for i in plant:
    shoes_produced[i] = sum([shoes_vars[i][j] for j in customer])

boots_produced = {}
for i in plant:
    boots_produced[i] = sum([boots_vars[i][j] for j in customer])

shoes_sent = {}
for i in customer:
    shoes_sent[i] = sum([shoes_vars[j][i] for j in plant])

boots_sent = {}
for i in customer:
    boots_sent[i] = sum([boots_vars[j][i] for j in plant])

    
#objective function

##fixed cost
fix_cost = 0
for i in plant[1:]:
    fix_cost += lpSum(y_vars[i][j] * df_fc[i][j] for j in prod_line)
fix_cost += 2500000 #stl fixed cost

##variable cost
prod_cost = 0
prod_cost += lpSum(shoes_produced[i] * shoes_vc[i] for i in plant) + lpSum(boots_produced[i] * boots_vc[i] for i in plant)

##transportation cost
trans_cost = 0
trans_cost += lpSum(df_distance[i][j] * shoes_vars[i][j] for i in plant for j in customer) * 0.03  #shoes
trans_cost += lpSum(df_distance[i][j] * boots_vars[i][j] for i in plant for j in customer) *0.04 #boots


probA += fix_cost + prod_cost + trans_cost

In [4]:
#constraints
##binary
probA += y_vars['KA'][1] + y_vars['KA'][2] == 1

for i in plant[2:]:
    probA += lpSum(y_vars[i][j] for j in prod_line) <= 1

#capacity
probA += shoes_produced['STL'] / 15000 + boots_produced['STL']/9000 <= 2

for i in plant[1:]:
    probA += shoes_produced[i] / 15000 + boots_produced[i] / 9000 <= lpSum(y_vars[i][j] * j for j in prod_line)

#demand
for i in customer:
    probA += shoes_sent[i] >= shoes_demand[i]

for i in customer:
    probA += boots_sent[i] >= boots_demand[i]


shoes:
MINIMIZE
20.4*boots_CO_COL + 25.2*boots_CO_KS1 + 25.2*boots_CO_KS2 + 26.8*boots_CO_SPR + 24.8*boots_CO_STL1 + 24.8*boots_CO_STL2 + 24.8*boots_CO_STL3 + 24.8*boots_CO_STL4 + 23.0*boots_IN_COL + 18.6*boots_IN_KS1 + 18.6*boots_IN_KS2 + 24.8*boots_IN_SPR + 27.6*boots_IN_STL1 + 27.6*boots_IN_STL2 + 27.6*boots_IN_STL3 + 27.6*boots_IN_STL4 + 30.2*boots_KA_COL + 25.4*boots_KA_KS1 + 25.4*boots_KA_KS2 + 31.6*boots_KA_SPR + 35.0*boots_KA_STL1 + 35.0*boots_KA_STL2 + 35.0*boots_KA_STL3 + 35.0*boots_KA_STL4 + 26.8*boots_SP_COL + 26.6*boots_SP_KS1 + 26.6*boots_SP_KS2 + 20.4*boots_SP_SPR + 28.6*boots_SP_STL1 + 28.6*boots_SP_STL2 + 28.6*boots_SP_STL3 + 28.6*boots_SP_STL4 + 34.8*boots_STL_COL + 40.0*boots_STL_KS1 + 40.0*boots_STL_KS2 + 38.6*boots_STL_SPR + 30.4*boots_STL_STL1 + 30.4*boots_STL_STL2 + 30.4*boots_STL_STL3 + 30.4*boots_STL_STL4 + 15.3*shoes_CO_COL + 18.9*shoes_CO_KS1 + 18.9*shoes_CO_KS2 + 20.1*shoes_CO_SPR + 18.6*shoes_CO_STL1 + 18.6*shoes_CO_STL2 + 18.6*shoes_CO_STL3 + 18.6*shoes_CO

In [5]:
probA.writeLP("ProbA.lp")
probA.solve()
print("Status:",LpStatus[probA.status])

print("Total cost =", value(probA.objective))

Status: Optimal
Total cost = 5990884.0


In [6]:
#a. production line decision output
output1 = []
for i in plant[1:]:
    line_output = []
    for j in prod_line:
        line_output.append(y_vars[i][j].varValue)
    output1.append(line_output)

output_df = pd.DataFrame(output1,index=plant[1:], columns=prod_line)
output_df

,1,2
KA,1.0,0.0
CO,0.0,0.0
SP,0.0,0.0
IN,0.0,1.0


In [7]:
#b. plants and the number of each product
output4 = []
for i in plant:
    line = []
    shoes = []
    boots = []
    for j in customer:
        shoes += [shoes_vars[i][j].varValue]
        boots += [boots_vars[i][j].varValue]
    line.append(sum(shoes))
    line.append(sum(boots))
    output4.append(line)

output_df4 = pd.DataFrame(output4,index=plant, columns=['shoes','boots'])
output_df4

,shoes,boots
STL,22000.0,4300.0
KA,0.0,6580.0
CO,0.0,0.0
SP,0.0,0.0
IN,24800.0,3120.0


In [8]:
#c. shoes output

output2 = []
for i in plant:
    line = []
    for j in customer:
        line.append(shoes_vars[i][j].varValue)
    line.append(sum(line))
    output2.append(line)

output_df2 = pd.DataFrame(output2,index=plant, columns=customer+['total'])
output_df2

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR,total
STL,8000.0,6500.0,7500.0,0.0,0.0,0.0,0.0,0.0,22000.0
KA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IN,0.0,0.0,0.0,0.0,9500.0,2300.0,7000.0,6000.0,24800.0


In [9]:
#c. boots output
output3 = []
for i in plant:
    line = []
    for j in customer:
        line.append(boots_vars[i][j].varValue)
    line.append(sum(line))
    output3.append(line)

output_df3 = pd.DataFrame(output3,index=plant, columns=customer+['total'])
output_df3

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR,total
STL,2200.0,0.0,1200.0,900.0,0.0,0.0,0.0,0.0,4300.0
KA,0.0,0.0,0.0,0.0,3380.0,0.0,0.0,3200.0,6580.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IN,0.0,0.0,0.0,0.0,1120.0,0.0,2000.0,0.0,3120.0


## problem2

In [10]:
probB=LpProblem("shoes2", LpMaximize)

In [11]:
#index
customer = ['STL1', 'STL2','STL3', 'STL4', 'KS1', 'KS2','COL','SPR']
plant = ['STL','KA','CO','SP','IN']

#parameters
d_shoes = [8000,6500,7500,0,9500,2300,7000,6000]
shoes_demand = dict(zip(customer,d_shoes))

d_boots = [2200,0,1200,900,4500,0,2000,3200]
boots_demand = dict(zip(customer,d_boots))

vc_shoes = [20,18,15,15,12]
shoes_vc = dict(zip(plant,vc_shoes))

vc_boots = [30,25,20,20,18]
boots_vc = dict(zip(plant,vc_boots))


trans_dis = [[10,10,10,10,250,250,120,215],\
             [250,250,250,250,10,10,130,165],\
             [120,120,120,120,130,130,10,170],\
             [215,215,215,215,165,165,170,10],\
             [240,240,240,240,15,15,125,170]]
df_distance = makeDict([plant,customer],trans_dis)

prod_line = [1,2]
fc = [[1000000,1700000],[1000000,1500000],[1000000,1500000],[900000,1300000]]
df_fc = makeDict([plant[1:],prod_line],fc)


#decision variables
y_vars = LpVariable.dicts("y",(plant[1:],prod_line),lowBound=0,cat='Binary')
shoes_vars = LpVariable.dicts("shoes",(plant,customer),lowBound=0,cat='Continuous')
boots_vars = LpVariable.dicts("boots",(plant,customer),lowBound=0,cat='Continuous')

shoes_produced = {}
for i in plant:
    shoes_produced[i] = sum([shoes_vars[i][j] for j in customer])

boots_produced = {}
for i in plant:
    boots_produced[i] = sum([boots_vars[i][j] for j in customer])

shoes_sent = {}
for i in customer:
    shoes_sent[i] = sum([shoes_vars[j][i] for j in plant])

boots_sent = {}
for i in customer:
    boots_sent[i] = sum([boots_vars[j][i] for j in plant])

    
#objective function

##fixed cost
fix_cost = 0
for i in plant[1:]:
    fix_cost += lpSum(y_vars[i][j] * df_fc[i][j] for j in prod_line)
fix_cost += 2500000 #stl fixed cost

##variable cost
prod_cost = 0
prod_cost += lpSum(shoes_produced[i] * shoes_vc[i] for i in plant) + lpSum(boots_produced[i] * boots_vc[i] for i in plant)

##transportation cost
trans_cost = 0
trans_cost += lpSum(df_distance[i][j] * shoes_vars[i][j] for i in plant for j in customer) * 0.03  #shoes
trans_cost += lpSum(df_distance[i][j] * boots_vars[i][j] for i in plant for j in customer) *0.04 #boots

##revenue
revenue = 0
revenue += 50 * lpSum(shoes_produced) + 80 * lpSum(boots_produced)

probB += revenue - (fix_cost + prod_cost + trans_cost)

In [12]:
#constraints
##binary
probB += y_vars['KA'][1] + y_vars['KA'][2] == 1

for i in plant[2:]:
    probB += lpSum(y_vars[i][j] for j in prod_line) <= 1

#capacity
probB += shoes_produced['STL'] / 15000 + boots_produced['STL']/9000 <= 2

for i in plant[1:]:
    probB += shoes_produced[i] / 15000 + boots_produced[i] / 9000 <= lpSum(y_vars[i][j] * j for j in prod_line)

#demand
for i in customer:
    probB += shoes_sent[i] == shoes_demand[i]

for i in customer:
    probB += boots_sent[i] == boots_demand[i]

shoes2:
MAXIMIZE
59.6*boots_CO_COL + 54.8*boots_CO_KS1 + 54.8*boots_CO_KS2 + 53.2*boots_CO_SPR + 55.2*boots_CO_STL1 + 55.2*boots_CO_STL2 + 55.2*boots_CO_STL3 + 55.2*boots_CO_STL4 + 57.0*boots_IN_COL + 61.4*boots_IN_KS1 + 61.4*boots_IN_KS2 + 55.2*boots_IN_SPR + 52.4*boots_IN_STL1 + 52.4*boots_IN_STL2 + 52.4*boots_IN_STL3 + 52.4*boots_IN_STL4 + 49.8*boots_KA_COL + 54.6*boots_KA_KS1 + 54.6*boots_KA_KS2 + 48.4*boots_KA_SPR + 45.0*boots_KA_STL1 + 45.0*boots_KA_STL2 + 45.0*boots_KA_STL3 + 45.0*boots_KA_STL4 + 53.2*boots_SP_COL + 53.4*boots_SP_KS1 + 53.4*boots_SP_KS2 + 59.6*boots_SP_SPR + 51.4*boots_SP_STL1 + 51.4*boots_SP_STL2 + 51.4*boots_SP_STL3 + 51.4*boots_SP_STL4 + 45.2*boots_STL_COL + 40.0*boots_STL_KS1 + 40.0*boots_STL_KS2 + 41.4*boots_STL_SPR + 49.6*boots_STL_STL1 + 49.6*boots_STL_STL2 + 49.6*boots_STL_STL3 + 49.6*boots_STL_STL4 + 34.7*shoes_CO_COL + 31.1*shoes_CO_KS1 + 31.1*shoes_CO_KS2 + 29.9*shoes_CO_SPR + 31.4*shoes_CO_STL1 + 31.4*shoes_CO_STL2 + 31.4*shoes_CO_STL3 + 31.4*shoes_C

In [13]:
probB.writeLP("ProbB.lp")
probB.solve()
print("Status:",LpStatus[probB.status])

print("Total profit =", value(probB.objective))

Status: Optimal
Total profit = -2530884.0


In [14]:
#a. production line decision output
output1_1 = []
for i in plant[1:]:
    line_output = []
    for j in prod_line:
        line_output.append(y_vars[i][j].varValue)
    output1_1.append(line_output)

output_df1_1 = pd.DataFrame(output1_1,index=plant[1:], columns=prod_line)
output_df1_1

,1,2
KA,1.0,0.0
CO,0.0,0.0
SP,0.0,0.0
IN,0.0,1.0


In [15]:
#b. plants and the number of each product
output2_1 = []
for i in plant:
    line = []
    shoes = []
    boots = []
    for j in customer:
        shoes += [shoes_vars[i][j].varValue]
        boots += [boots_vars[i][j].varValue]
    line.append(sum(shoes))
    line.append(sum(boots))
    output2_1.append(line)

output_df2_1 = pd.DataFrame(output2_1,index=plant, columns=['shoes','boots'])
output_df2_1

,shoes,boots
STL,22000.0,4300.0
KA,0.0,6580.0
CO,0.0,0.0
SP,0.0,0.0
IN,24800.0,3120.0


In [16]:
#c. shoes output

output3_1 = []
for i in plant:
    line = []
    for j in customer:
        line.append(shoes_vars[i][j].varValue)
    line.append(sum(line))
    output3_1.append(line)

output_df3_1 = pd.DataFrame(output3_1,index=plant, columns=customer+['total'])
output_df3_1

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR,total
STL,8000.0,6500.0,7500.0,0.0,0.0,0.0,0.0,0.0,22000.0
KA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IN,0.0,0.0,0.0,0.0,9500.0,2300.0,7000.0,6000.0,24800.0


In [17]:
#c. boots output
output4_1 = []
for i in plant:
    line = []
    for j in customer:
        line.append(boots_vars[i][j].varValue)
    line.append(sum(line))
    output4_1.append(line)

output_df4_1 = pd.DataFrame(output4_1,index=plant, columns=customer+['total'])
output_df4_1

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR,total
STL,2200.0,0.0,1200.0,900.0,0.0,0.0,0.0,0.0,4300.0
KA,0.0,0.0,0.0,0.0,3380.0,0.0,0.0,3200.0,6580.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IN,0.0,0.0,0.0,0.0,1120.0,0.0,2000.0,0.0,3120.0


## problem3

In [18]:
probC=LpProblem("shoes3", LpMaximize)

In [19]:
#index
customer = ['STL1', 'STL2','STL3', 'STL4', 'KS1', 'KS2','COL','SPR']
plant = ['STL','KA','CO','SP','IN']

#parameters
d_shoes = [8000,6500,7500,0,9500,2300,7000,6000]
shoes_demand = dict(zip(customer,d_shoes))

d_boots = [2200,0,1200,900,4500,0,2000,3200]
boots_demand = dict(zip(customer,d_boots))

vc_shoes = [20,18,15,15,12]
shoes_vc = dict(zip(plant,vc_shoes))

vc_boots = [30,25,20,20,18]
boots_vc = dict(zip(plant,vc_boots))


trans_dis = [[10,10,10,10,250,250,120,215],\
             [250,250,250,250,10,10,130,165],\
             [120,120,120,120,130,130,10,170],\
             [215,215,215,215,165,165,170,10],\
             [240,240,240,240,15,15,125,170]]
df_distance = makeDict([plant,customer],trans_dis)

prod_line = [1,2]
fc = [[1000000,1700000],[1000000,1500000],[1000000,1500000],[900000,1300000]]
df_fc = makeDict([plant[1:],prod_line],fc)


#decision variables
y_vars = LpVariable.dicts("y",(plant[1:],prod_line),lowBound=0,cat='Binary')
shoes_vars = LpVariable.dicts("shoes",(plant,customer),lowBound=0,cat='Continuous')
boots_vars = LpVariable.dicts("boots",(plant,customer),lowBound=0,cat='Continuous')

shoes_produced = {}
for i in plant:
    shoes_produced[i] = sum([shoes_vars[i][j] for j in customer])

boots_produced = {}
for i in plant:
    boots_produced[i] = sum([boots_vars[i][j] for j in customer])

shoes_sent = {}
for i in customer:
    shoes_sent[i] = sum([shoes_vars[j][i] for j in plant])

boots_sent = {}
for i in customer:
    boots_sent[i] = sum([boots_vars[j][i] for j in plant])

    
#objective function

##fixed cost
fix_cost = 0
for i in plant[1:]:
    fix_cost += lpSum(y_vars[i][j] * df_fc[i][j] for j in prod_line)
fix_cost += 2500000 #stl fixed cost

##variable cost
prod_cost = 0
prod_cost += lpSum(shoes_produced[i] * shoes_vc[i] for i in plant) + lpSum(boots_produced[i] * boots_vc[i] for i in plant)

##transportation cost
trans_cost = 0
trans_cost += lpSum(df_distance[i][j] * shoes_vars[i][j] for i in plant for j in customer) * 0.03  #shoes
trans_cost += lpSum(df_distance[i][j] * boots_vars[i][j] for i in plant for j in customer) *0.04 #boots

##revenue
revenue = 0
revenue += 50 * lpSum(shoes_produced) + 80 * lpSum(boots_produced)

probC += revenue - (fix_cost + prod_cost + trans_cost)

In [20]:
#constraints
##binary
probC += y_vars['KA'][1] + y_vars['KA'][2] == 1

for i in plant[2:]:
    probC += lpSum(y_vars[i][j] for j in prod_line) <= 1

#capacity
probC += shoes_produced['STL'] / 15000 + boots_produced['STL']/9000 <= 2

for i in plant[1:]:
    probC += shoes_produced[i] / 15000 + boots_produced[i] / 9000 <= lpSum(y_vars[i][j] * j for j in prod_line)

#demand
for i in customer:
    probC += shoes_sent[i] <= shoes_demand[i]

for i in customer:
    probC += boots_sent[i] <= boots_demand[i]



In [21]:
probC.writeLP("ProbC.lp")
probC.solve()
print("Status:",LpStatus[probC.status])

print("Total profit =", value(probC.objective))

Status: Optimal
Total profit = -2127270.0


In [22]:
#a. production line decision output
output1_2 = []
for i in plant[1:]:
    line_output = []
    for j in prod_line:
        line_output.append(y_vars[i][j].varValue)
    output1_2.append(line_output)

output_df1_2 = pd.DataFrame(output1_2,index=plant[1:], columns=prod_line)
output_df1_2

,1,2
KA,1.0,0.0
CO,0.0,0.0
SP,0.0,0.0
IN,0.0,0.0


In [23]:
#b. plants and the number of each product
output2_2 = []
for i in plant:
    line = []
    shoes = []
    boots = []
    for j in customer:
        shoes += [shoes_vars[i][j].varValue]
        boots += [boots_vars[i][j].varValue]
    line.append(sum(shoes))
    line.append(sum(boots))
    output2_2.append(line)

output_df2_2 = pd.DataFrame(output2_2,index=plant, columns=['shoes','boots'])
output_df2_2

,shoes,boots
STL,22000.0,4800.0
KA,7500.0,4500.0
CO,0.0,0.0
SP,0.0,0.0
IN,0.0,0.0


In [24]:
#c. shoes output

output3_2 = []
for i in plant:
    line = []
    for j in customer:
        line.append(shoes_vars[i][j].varValue)
    line.append(sum(line))
    output3_2.append(line)

output_df3_2 = pd.DataFrame(output3_2,index=plant, columns=customer+['total'])
output_df3_2

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR,total
STL,8000.0,6500.0,7500.0,0.0,0.0,0.0,0.0,0.0,22000.0
KA,0.0,0.0,0.0,0.0,5200.0,2300.0,0.0,0.0,7500.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
#c. boots output
output4_2 = []
for i in plant:
    line = []
    for j in customer:
        line.append(boots_vars[i][j].varValue)
    line.append(sum(line))
    output4_2.append(line)

output_df4_2 = pd.DataFrame(output4_2,index=plant, columns=customer+['total'])
output_df4_2

,STL1,STL2,STL3,STL4,KS1,KS2,COL,SPR,total
STL,2200.0,0.0,1200.0,900.0,0.0,0.0,500.0,0.0,4800.0
KA,0.0,0.0,0.0,0.0,4500.0,0.0,0.0,0.0,4500.0
CO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
